<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Stefano Schiappacasse


### **Link de repositorio de GitHub:** `https://github.com/stefanoschiappacasse/RepoMDS7202`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

#Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, TransformerMixin

#Clasificadores
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

#Regresores
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor



# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score

#Libreria para plotear
# !pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
# !pip install umap-learn

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
df_players = pd.read_csv('stats_players.csv')
df_players.head()

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [3]:
### Código aquí ###
df_players['SeleccionadoNacional'] =  df_players['National_Position'].apply(lambda x: 0 if pd.isna(x) else 1)
df_players['SeleccionadoNacional'].value_counts()

0    16513
1     1075
Name: SeleccionadoNacional, dtype: int64

Se observa que existen muchos más jugadores que no son seleccionados nacionales. Esto tiene sentido ya que por cada país, existen muchos jugadores, pero sólo unos pocos son seleccionados para jugar por su selección.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="http://vignette3.wikia.nocookie.net/inciclopedia/images/c/cd/Patada_Medel_Messi.gif/revision/latest?cb=20150705215005" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [4]:
#columnas categóricas a transformar
columnasCategoricas = ['Nationality', 'Club_Position']
#columnas numéricas a transformar
columnasNumericas = ['Height', 'Weight', 'Age', 'Ball_Control',
                  'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
                  'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
                  'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
                  'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
                  'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
                  'Penalties', 'Volleys', 'WorkRate_Ataque', 'WorkRate_Defensa']
#esta función se crea para realizar un preprocesamiento previo antes de transformar las variables
def preprocesar(data):
    df = data.copy()
    #Eliminar columnas que no se utilizarán para la predicción. 
    df.drop(columns = ['Name', 'National_Position'], inplace = True, errors = 'ignore')
    #Se imputan los valores nulos para la varible Club_Position
    if df.columns.isin(['Club_Position']).sum() != 0:
        df['Club_Position'] = df['Club_Position'].fillna(value = 'SinClub')
    #Se codifica el pie del jugador como una variable binaria
    df['Preffered_Foot'] = df['Preffered_Foot'].map({'Right': 0, 'Left': 1})
    #Se separa el atributo Work_Rate en dos, ya que contiene información sobre la actitud de
    #ataque y de defensa del jugador.
    wr = df['Work_Rate'].str.split(pat = ' / ', expand = True)\
         .rename(columns = {0: 'WorkRate_Ataque', 1: 'WorkRate_Defensa'})
    #Se mapean los valores de los Work_Rates de manera ordinal (no lo realizo en el column transformer ya que 
    #los valores de una transformación no se pasan a la siguiente)
    wr['WorkRate_Ataque'] = wr['WorkRate_Ataque'].map({'Low': 0, 'Medium': 1, 'High': 2})
    wr['WorkRate_Defensa'] = wr['WorkRate_Defensa'].map({'Low': 0, 'Medium': 1, 'High': 2})
    
    #Se retorna el dataframe preprocesado
    return pd.concat([df.drop(columns = ['Work_Rate']), wr], axis = 1)

#column transformer que aplica transformaciones a las variables categóricas y numéricas.
transformacionColumnas = ColumnTransformer(transformers = 
                       [
                           ('OneHot', OneHotEncoder(sparse= False,handle_unknown ='ignore',), columnasCategoricas),
                           ('MinMax', MinMaxScaler(), columnasNumericas),
                       ],
                       remainder = 'passthrough',
                       verbose_feature_names_out=False)


#Pipe con support vector machine
pipeLinearSVC = Pipeline([
    ('Procesamiento', FunctionTransformer(preprocesar)),
    ('Transformacion', transformacionColumnas),
    ('Clasificador', LinearSVC(random_state = 100))
])

#Pipe con KNN
pipeKNN = Pipeline([
    ('Procesamiento', FunctionTransformer(preprocesar)),
    ('Transformacion', transformacionColumnas),
    ('Clasificador', KNeighborsClassifier())
])

#Pipe con clasificador naive bayes gaussiano
pipeRFC = Pipeline([
    ('Procesamiento', FunctionTransformer(preprocesar)),
    ('Transformacion', transformacionColumnas),
    ('Clasificador', RandomForestClassifier(random_state = 100))
])

A continuación se justifican las distintas transformaciones realizadas a los distintos atributos:
1) Se decide eliminar la columna Nombre debido a que no entrega mucha información y no se utilizará para predecir. También se decide eliminar National_Position, ya que permite identificar claramente qué jugador es seleccionado nacional o no.

2) Las columnas a las cuales se le aplica la transformación de onehotencoder son aquellas de tipo categoría, y que no tienen un orden en intrínsico.

3) La columna Preffered_Foot se decide cambiar por una columna binaria, debido a que posee sólo dos posibles valores.

4) La columna Work_Rate se decide separar en dos, debido a que contiene información sobre el desempeño en ataque y en defensa del jugador.

5) Las variables resultantes de la separación de Work_Rate son de tipo ordinal, por lo que se las cambiará por variables numéricas y se preservará ese orden.

6) A todas las columnas que son numéricas, inclusive las resultantes de la transformación ordinal, se les aplicará una transformación Min-Max para que queden en el mismo rango de valores. Esta decisión proviene luego de haber observado la distribución de éstas, en las cuales no se aprecian outliers y así todas éstas estarán en la misma escala.

7) Por último, la elección de los clasificadores se basó en la guía de sklearn. Primero se eligió el clasificador LinearSVC ya que es el primero recomendado para clasificación con menos de 100 mil registros, luego se entrenará un KNN que es el siguiente recomendado cuando los datos no son de texto, y por último se ajustará un Random Forest que es un tipo de clasificador de ensamble, recomendado cuando el KNN no tiene buenos resultados

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [7]:
#Separación de conjunto de entrenamiento y testeo.
features = df_players.drop(columns = ['SeleccionadoNacional'])
labels = df_players['SeleccionadoNacional']
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels, random_state = 100
)

#función que permite entrenar y mostrar el desempeño del modelo
def train_and_evaluate(pipe,
                       X_train,
                       y_train,
                       X_test,
                       y_test,
                       tipo,
                       print_=True):

    # notar que los datasets son parámetros por defecto.
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    if print_:
        if tipo in ['clas_binaria', 'clas_multiclase']:
            if tipo == 'clas_binaria':
                print('Matriz de confusión: \n')
                print(confusion_matrix(y_test, y_pred))
                print('\nReporte de Clasificación: \n')
                print(classification_report(y_test, y_pred))
            else:
                print('Matriz de confusión: \n')
                print(confusion_matrix(y_test, y_pred, labels=pipe.classes_))
                print('\nReporte de Clasificación: \n')
                print(classification_report(y_test, y_pred, labels=pipe.classes_))
        else:
            print('R2: \n')
            print(r2_score(y_test, y_pred))
    
    return 

In [9]:
#Entrenamiento con el árbol de decisión
train_and_evaluate(pipeLinearSVC,
                   X_train,
                   y_train,
                   X_test,
                   y_test, 
                   tipo = 'clas_binaria')

Matriz de confusión: 

[[5383   67]
 [ 216  139]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      5450
           1       0.67      0.39      0.50       355

    accuracy                           0.95      5805
   macro avg       0.82      0.69      0.73      5805
weighted avg       0.94      0.95      0.95      5805



D:\anaconda3\envs\MDS7202\lib\site-packages\sklearn\svm\_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [10]:
#Entrenamiento con el support vector machine
train_and_evaluate(pipeKNN,
                   X_train,
                   y_train,
                   X_test,
                   y_test, 
                   tipo = 'clas_binaria')

Matriz de confusión: 

[[5423   27]
 [ 311   44]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5450
           1       0.62      0.12      0.21       355

    accuracy                           0.94      5805
   macro avg       0.78      0.56      0.59      5805
weighted avg       0.93      0.94      0.92      5805



In [11]:
#Entrenamiento con Naive Bayes gaussiano
train_and_evaluate(pipeRFC,
                   X_train,
                   y_train,
                   X_test,
                   y_test, 
                   tipo = 'clas_binaria')

Matriz de confusión: 

[[5433   17]
 [ 315   40]]

Reporte de Clasificación: 

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5450
           1       0.70      0.11      0.19       355

    accuracy                           0.94      5805
   macro avg       0.82      0.55      0.58      5805
weighted avg       0.93      0.94      0.92      5805



**Respuesta**:
Al observar los resultados para cada modelo, especialmente el F1, podemos notar que en general no les va muy bien identificando la clase positiva, es decir que un jugador sea seleccionado nacional, pero sí todos tienen muy buen desempeño al momento de clasificar la clase negativa. El que tiene mejores resultados con la clase positiva es la support vector machine lineal, que obtiene un F1 de 0.5, los dos otros modelos poseen aproximadamente 0.2 para esta métrica. Si miramos ahora la precisión de la clase positiva, podemos observar que todos los modelos obtienen un valor entre 0.6 y 0.7, sin embargo donde se caen es en el recall, en donde el mejor resultado lo tiene la SVMLineal con un valor de 0.39, lo que indica que ningún modelo es capaz de identificar de buena manera a todos los seleccionados nacionales. 

Estos resultados podrían mejorarse si realizaramos algo de ingeniería de características y aumentemos la cantidad de datos que tengamos. 

La cantidad de ejemplos por clase también influye en los resultados ya que al tener un desbalance de clases, el clasificador tenderá a predecir la clase mayoritaria ya que eso mejorará su desempeño global y, por otro lado, tendrá menos ejemplos de la clase minoritaria que le dificultará el aprendizaje sobre ésta. Si realmente sólo nos interesa la clase positiva, se podría establecer una matriz de costos que penalice los errores cometidos en esta clase, y así se elija un modelo que minimice el costo, y no sólo máxime el desempeño respecto a alguna métrica en particular.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [12]:
#Mapeo de valores establecidos en el enunciado
dictClubPosition = {'ST':'ataque',
                    'CF':'ataque',
                    'RW':'central_ataque',
                    'CAM':'central_ataque',
                    'LW':'central_ataque',
                    'RM':'central',
                    'CM':'central',
                    'LM':'central',
                    'RWB':'central_defensa',
                    'CDM':'central_defensa',
                    'LWB':'central_defensa',
                    'RB':'defensa',
                    'CB':'defensa',
                    'LB':'defensa',
                    'GK':'arquero'}
#Se eliminan las columnas National_Position y SeleccionadoNacional debido a que así fue indicado por el profesor. 
#La razón es porque entrega demasiada información sobre la variable predictora.
df_players2 = df_players.drop(columns = ['National_Position', 'SeleccionadoNacional'])
#Se mapean los valores de la variable a predecir
df_players2['Club_Position'] = df_players['Club_Position'].map(dictClubPosition)
#Se eliminan los valores nulos luego de hacer el mapeo, ya que existen muchas posiciones que no fueron mapeadas.
df_players2.dropna(inplace = True)
#Casos por clase
df_players2['Club_Position'].value_counts()

Se puede observar que luego de hacer el mapeo de la variable `Club_Position`, las 3 posiciones que más jugadores poseen son la de defensa, central y arquero. En cambio, las posiciones central ataque, ataque y central defensa son las que menos se repiten en el dataset. Notar que existen casi 6 veces más defensas que central defensas. Este desbalance de clases podría generar que el modelo no se comporte bien con las clases minoritarias.

In [14]:
#Se Crea una nueva lista de columnas categóricas para este ejercicio,
#ya que ahora no se utilizará la variable Club_Position
columnasCategoricas2 = ['Nationality']
#Nueva transformación de columnas
transformacionColumnas2 = ColumnTransformer(transformers = 
                       [
                           ('OneHot', OneHotEncoder(sparse= False,handle_unknown ='ignore',), columnasCategoricas2),
                           ('MinMax', MinMaxScaler(), columnasNumericas),
                       ],
                       remainder = 'passthrough',
                       verbose_feature_names_out=False
                                          )


#Pipe con support vector machine lineal
pipeLinearSVC2 = Pipeline([
    ('Procesamiento', FunctionTransformer(preprocesar)),
    ('Transformacion', transformacionColumnas2),
    ('Clasificador', LinearSVC(random_state = 100))
])

#Pipe con KNN
pipeKNN2 = Pipeline([
    ('Procesamiento', FunctionTransformer(preprocesar)),
    ('Transformacion', transformacionColumnas2),
    ('Clasificador', KNeighborsClassifier())
])

#Pipe con clasificador de ensamble random forest
pipeRFC2 = Pipeline([
    ('Procesamiento', FunctionTransformer(preprocesar)),
    ('Transformacion', transformacionColumnas2),
    ('Clasificador', RandomForestClassifier(random_state = 100))
])

#Separación de conjunto de datos
features2 = df_players2.drop(columns = ['Club_Position'])
labels2 = df_players2['Club_Position']
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    features2, labels2, test_size=0.33, shuffle=True, stratify=labels2, random_state = 100
)

In [15]:
#Entrenamiento y evaluación del modelo SVC lineal
train_and_evaluate(pipeLinearSVC2,
                    X_train2,
                    y_train2,
                    X_test2,
                    y_test2,
                    'clas_multiclase')

Matriz de confusión: 

[[209   0   0   0   0   0]
 [  0 116  15   9   0   2]
 [  0  15 202  35   5  42]
 [  0  16 112  51   4   9]
 [  0   0   6   2  14  47]
 [  0   0  12   3   3 371]]

Reporte de Clasificación: 

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.79      0.82      0.80       142
        central       0.58      0.68      0.63       299
 central_ataque       0.51      0.27      0.35       192
central_defensa       0.54      0.20      0.29        69
        defensa       0.79      0.95      0.86       389

       accuracy                           0.74      1300
      macro avg       0.70      0.65      0.66      1300
   weighted avg       0.72      0.74      0.72      1300



D:\anaconda3\envs\MDS7202\lib\site-packages\sklearn\svm\_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [16]:
#Entrenamiento y evaluación del modelo KNN
train_and_evaluate(pipeKNN2,
                    X_train2,
                    y_train2,
                    X_test2,
                    y_test2,
                    'clas_multiclase')

Matriz de confusión: 

[[209   0   0   0   0   0]
 [  0  88  42   8   0   4]
 [  0  42 148  49   5  55]
 [  0  29  94  48   2  19]
 [  0   2  14   1   6  46]
 [  0   2  44   4   8 331]]

Reporte de Clasificación: 

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.54      0.62      0.58       142
        central       0.43      0.49      0.46       299
 central_ataque       0.44      0.25      0.32       192
central_defensa       0.29      0.09      0.13        69
        defensa       0.73      0.85      0.78       389

       accuracy                           0.64      1300
      macro avg       0.57      0.55      0.55      1300
   weighted avg       0.62      0.64      0.62      1300



In [17]:
#Entrenamiento y evaluación del random forest
train_and_evaluate(pipeRFC2,
                    X_train2,
                    y_train2,
                    X_test2,
                    y_test2,
                    'clas_multiclase')

Matriz de confusión: 

[[209   0   0   0   0   0]
 [  0 115  24   2   0   1]
 [  0  10 215  34   5  35]
 [  0  12 123  49   1   7]
 [  0   0   4   1   6  58]
 [  0   0  11   1   5 372]]

Reporte de Clasificación: 

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.84      0.81      0.82       142
        central       0.57      0.72      0.64       299
 central_ataque       0.56      0.26      0.35       192
central_defensa       0.35      0.09      0.14        69
        defensa       0.79      0.96      0.86       389

       accuracy                           0.74      1300
      macro avg       0.69      0.64      0.64      1300
   weighted avg       0.72      0.74      0.71      1300



**Respuesta**:
Se descatan varios aspectos de los resultados obtenidos. Primero 1) notar que las clases que obtuvieron peores desempeños en todos lo modelos fueron la clase `central_ataque` y `central_defensa`. Destacar que estas clases están entre las 3 clases que tienen menor cantidad de ejemplos y que ambas son mezclas de otras clases, que son `central`, `defensa` y `ataque`, es aspecto podría indicar que los jugadores de estas clases mezcladas tienen características muy similares a los jugadores de sus clases "raíces", lo que haría que el modelo se confunda al momento de hacer la predicción. Lo segundo 2) notar que el ranking de desempeño de los modelos mantuvo el orden que se obtuvo en el primer ejercicio de clasificación binaria, siendo la SVCLinear la que mejor se comportó, seguida por el Random Forest y por último el KNN. Por otro lado, 3) es importante notar que a pesar de que la clase de `ataque` tenía menos ejemplos, en todos los modelos obtuvo mejores resultados que la clase `central_ataque`, lo que deja entrever que no hay una relación directa entre la cantidad de ejemplos y el desempeño del modelo. Un ejemplo más claro es el de la clase `arquero` el cual tiene practicamente la mitad de ejemplos que la clase mayoritaria, y los modelos son capaces de reconocerla sin problemas, teniendo un 1 en todas sus métricas, a diferencia de la clase `defensa` la cual a pesar de ser la clase con segundo mejor desempeño, no supera un F1 de 0.86.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

La métrica $R^2$ es un indicador de qué tanta variabilidad de los datos es capaz de explicar el modelo propuesto. Su fórmula es $R^2 = 1 - \frac{SSE}{SST}$ en donde $SSE$ es la suma del error cuadrático, en donde el error es la desviación de la predicción del modelo respecto al valor real de la observación, y el $SST$ es lo mismo que el $SSE$ pero ahora asumiendo que mi predicción sea el promedio. Entonces básicamente este indicador mide qué tan bien lo hace nuestro modelo respecto a la si nuestra predicción sea simplemente el promedio. Si mi modelo no lo hace mucho mejor, entonces el $R2$ será bajo, y si lo hace mucho mejor, es decir su $SSE$ es bajo, entonces tendrá un $R2 alto.

In [18]:
#cargar datos
sueldos = pd.read_csv('salarios.csv')
#eliminar primera columna sin información
sueldos.drop(columns = ['Unnamed: 0'], inplace = True)
#unión datos de sueldos con las características de los jugadores. 
consolidado = pd.merge(left = sueldos,
                       right = df_players.drop(columns = ['SeleccionadoNacional']), 
                       how = 'inner', left_on = 'Player', right_on = 'Name')
#Eliminar la columna "player" ya que contiene la misma información que "name"
consolidado.drop(columns = ['Player'], inplace = True)
#separación de conjunto en train y test
features3 = consolidado.drop(columns = ['Weekly Salary'])
labels3 = consolidado['Weekly Salary']
X_train3, X_test3, y_train3, y_test3 = train_test_split(
    features3, labels3, test_size=0.33, shuffle=True, random_state = 100
)
#pipeline para la regresión ridge
pipeRidge = Pipeline([
    ('Procesamiento', FunctionTransformer(preprocesar)),
    ('Transformacion', transformacionColumnas),
    ('Clasificador', Ridge(random_state = 100))
])
#pipeline para la support vector machine regressor
pipeSVR = Pipeline([
    ('Procesamiento', FunctionTransformer(preprocesar)),
    ('Transformacion', transformacionColumnas),
    ('Clasificador', SVR(kernel = 'rbf'))
])
#pipeline para el random forest regressor
pipeRFR = Pipeline([
    ('Procesamiento', FunctionTransformer(preprocesar)),
    ('Transformacion', transformacionColumnas),
    ('Clasificador', RandomForestRegressor(random_state = 100))
])

In [19]:
train_and_evaluate(pipeRidge,
                    X_train3,
                    y_train3,
                    X_test3,
                    y_test3,
                    'regresion')

R2: 

0.21303206582782908


In [20]:
train_and_evaluate(pipeSVR,
                    X_train3,
                    y_train3,
                    X_test3,
                    y_test3,
                    'regresion')

R2: 

-0.15888492748840077


In [21]:
train_and_evaluate(pipeRFR,
                    X_train3,
                    y_train3,
                    X_test3,
                    y_test3,
                    'regresion')

R2: 

0.4731212830863586


**Respuesta**:
Los resultados muestran que el modelo de random forest es el que mejor desempeño obtuvo en esta labor predecir el salario del jugador, con un $R2$ de 0.47, que indica que sólo el 47% de la variabilidad inherente a los datos es capaz de explicarla el modelo. Otro aspecto relevante a considerar es que el modelo de la SVR obtuvo un $R2$ negativo, indicando que si se utilizara el promedio de todos los salarios para predecir el salario de un jugador tendríamos mejores resultados que si usamos el modelo. Para el caso de la regresión de Rigde, si bien sus resultados no fueron buenos, por lo menos algo de la varianza es capaz de explicar.

Estos resultados se pueden mejorar aumentando la cantidad de ejemplo, capturando otras características o haciendo ingeniería de las que tenemos. Más ejemplos siempre permitirán a los modelos aprender mejor de los datos, pero no siempre agregar nuevas features mejorará el desempeño. La ingeniería de características tampoco garantiza mejores resultados, pero probablemente sí lo haga. Por lo tanto, lo ideal sería obtener más ejemplos, e ir probando y agregar  nuevas características y también hacer ingeniería sobre éstas, cómo por ejemplo, la cantidad de títulos en los últimos 5 años.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>